In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats


In [2]:
import sys
print(sys.executable)
print(sys.version)

c:\Users\recepcion.indisaprov\AppData\Local\Programs\Python\Python312\python.exe
3.12.5 (tags/v3.12.5:ff3bc82, Aug  6 2024, 20:45:27) [MSC v.1940 64 bit (AMD64)]


In [3]:
# Instala streamlit en el mismo entorno del kernel
%pip install --upgrade pip
%pip install streamlit

# Verificar importación
import streamlit as st
print("streamlit instalado:", st.__version__)

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
streamlit instalado: 1.52.1


In [4]:
# Instala plotly en el mismo entorno del kernel
%pip install --upgrade pip
%pip install plotly

# Verificar importación
import plotly
print("plotly instalado:", plotly.__version__)


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
plotly instalado: 6.5.0


In [5]:
import streamlit as st
import pandas as pd
import plotly.express as px

st.set_page_config(page_title="CaféData", layout="wide")

st.title("☕ CaféData")
st.subheader("Análisis simple para cafeterías y tiendas")

# --- Cargar archivo ---
archivo = st.file_uploader(
    "ventas_cafeteria.xlsx",
    type=["xlsx", "csv"]
)

if archivo is not None:
    df = pd.read_excel("ventas_cafeteria.xlsx")

    # --- Validación mínima ---
    columnas_requeridas = {
        "fecha", "producto", "categoria",
        "cantidad", "precio_unitario", "costo_unitario"
    }

    if not columnas_requeridas.issubset(df.columns):
        st.error("❌ El archivo no tiene las columnas necesarias")
        st.stop()

    # --- Procesamiento ---
    df["fecha"] = pd.to_datetime(df["fecha"])
    df["ingresos"] = df["cantidad"] * df["precio_unitario"]
    df["costo_total"] = df["cantidad"] * df["costo_unitario"]
    df["ganancia"] = df["ingresos"] - df["costo_total"]

    # --- KPIs ---
    col1, col2, col3 = st.columns(3)
    col1.metric("💰 Ingresos totales", f"${df['ingresos'].sum():,.0f}")
    col2.metric("📦 Ganancia total", f"${df['ganancia'].sum():,.0f}")
    col3.metric("📈 Margen promedio", f"{(df['ganancia'].sum()/df['ingresos'].sum())*100:.1f}%")

    # --- Gráfico 1: Ventas por día ---
    ventas_dia = df.groupby("fecha")["ingresos"].sum().reset_index()
    fig1 = px.line(ventas_dia, x="fecha", y="ingresos", title="Ventas por día")
    st.plotly_chart(fig1, use_container_width=True)

    # --- Gráfico 2: Productos más rentables ---
    rentabilidad = (
        df.groupby("producto")["ganancia"]
        .sum()
        .sort_values(ascending=False)
        .reset_index()
    )
    fig2 = px.bar(
        rentabilidad.head(10),
        x="producto",
        y="ganancia",
        title="Productos más rentables"
    )
    st.plotly_chart(fig2, use_container_width=True)

    # --- Insight automático ---
    top_producto = rentabilidad.iloc[0]
    st.success(
        f"📌 Tu producto más rentable es **{top_producto['producto']}** "
        f"con ${top_producto['ganancia']:,.0f} de ganancia."
    )

else:
    st.info("👆 Sube un archivo para comenzar")


2025-12-19 08:24:09.080 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-19 08:24:09.080 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-19 08:24:10.230 
  command:

    streamlit run C:\Users\recepcion.indisaprov\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-12-19 08:24:10.231 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-19 08:24:10.231 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-19 08:24:10.233 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-19 08:24:10.236 Thread 'MainThread': missing ScriptRunContext! Thi

In [ ]:
streamlit run echo "# Mi Proyecto" >> README.md.py


SyntaxError: invalid syntax (507122745.py, line 1)